# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [92]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [93]:
# your code here
admissions_predict_df = pd.read_csv('/Users/amandine/Desktop/Ironhack/03_Week/lab-advanced-pandas/your-code/data/Admission_Predict.csv') #You can give an argument to read a particular table of the spread sheet
admissions_predict_df.head(3)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80


Remove trailing spaces at the end of the column names if there are any.


In [94]:
# Check for trailing spaces in column names
for column in admissions_predict_df.columns:
    if column.endswith(' '):
        print(f"Column name {column} has trailing spaces.")

Column name LOR  has trailing spaces.
Column name Chance of Admit  has trailing spaces.


In [95]:
# With list comprehensions
columns_with_trailing_spaces = [column for column in admissions_predict_df.columns if column.endswith(' ')]
print(f"Columns with trailing spaces: {columns_with_trailing_spaces}")

Columns with trailing spaces: ['LOR ', 'Chance of Admit ']


In [96]:
# your code here
admissions_predict_df.columns = admissions_predict_df.columns.str.rstrip()
admissions_predict_df.head(3)

#method with list comprehensions : 
#df.columns = [col.strip() for col in df.columns]
#.columns.str.rstrip(): Removes trailing spaces from all column names in the DataFrame, 
#modifying the original DataFrame's column names.
#
#col.strip(): Removes trailing spaces from an individual column name (string) when used within a loop, 
#but it doesn't modify the original DataFrame.

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80


Let's evaluate the dataset by looking at the `head` function.

In [97]:
# your code here
admissions_predict_df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [98]:
# your code here
missing_data = admissions_predict_df.isnull().sum()
print(missing_data)

# Display columns with missing data / other method
#columns_with_missing_data = missing_data[missing_data > 0]
#print(columns_with_missing_data)

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64


In [99]:
missing_data_2 = admissions_predict_df.isna().sum()
print(missing_data_2)

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [100]:
# your code here
#admissions_predict_df.set_index(['Serial No.', 'GRE Score', 'CGPA'], inplace=True)
#admissions_predict_df.head()

admissions_predict_df = admissions_predict_df.set_index('Serial No.')
admissions_predict_df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [101]:
# your code here

admissions_predict_df.set_index(['GRE Score', 'CGPA'], inplace=True, drop=False)
admissions_predict_df.head() 

,,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
GRE Score,CGPA,,,,,,,,
337,9.65,337,118,4,4.5,4.5,9.65,1,0.92
316,8.00,316,104,3,3.0,3.5,8.00,1,0.72
322,8.67,322,110,3,3.5,2.5,8.67,1,0.80
314,8.21,314,103,2,2.0,3.0,8.21,0,0.65
330,9.34,330,115,5,4.5,3.0,9.34,1,0.90


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [102]:
admissions_predict_df.set_index(['GRE Score', 'CGPA'], inplace=True, drop=True)
admissions_predict_df.head() 

,,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,
337,9.65,118,4,4.5,4.5,1,0.92
316,8.00,104,3,3.0,3.5,1,0.72
322,8.67,110,3,3.5,2.5,1,0.80
314,8.21,103,2,2.0,3.0,0,0.65
330,9.34,115,5,4.5,3.0,1,0.90


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [103]:
admissions_predict_df.reset_index(inplace=True)
admissions_predict_df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
1,316,8.00,104,3,3.0,3.5,1,0.72
2,322,8.67,110,3,3.5,2.5,1,0.80
3,314,8.21,103,2,2.0,3.0,0,0.65
4,330,9.34,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [104]:
# your code here
filtered_df = admissions_predict_df[(admissions_predict_df['CGPA'] > 9) & (admissions_predict_df['Research'] == 1)]
filtered_df.head(10)

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
4,330,9.34,115,5,4.5,3.0,1,0.90
10,328,9.10,112,4,4.0,4.5,1,0.78
19,328,9.50,116,5,5.0,5.0,1,0.94
20,334,9.70,119,5,5.0,4.5,1,0.95
21,336,9.80,119,5,4.0,3.5,1,0.97
28,338,9.40,118,4,3.0,4.5,1,0.91
29,340,9.60,114,5,4.0,4.0,1,0.90
30,331,9.80,112,5,4.0,5.0,1,0.94
31,320,9.20,110,5,5.0,5.0,1,0.88


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [105]:
# your code here
sop_score = filtered_df[(filtered_df['SOP'] < 3.5)]
sop_score.head(10)

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
28,338,9.40,118,4,3.0,4.5,1,0.91
140,326,9.11,114,3,3.0,3.0,1,0.83
217,324,9.01,111,4,3.0,3.0,1,0.82
381,325,9.11,107,3,3.0,3.5,1,0.84


In [106]:
mean_chance_of_admit = sop_score['Chance of Admit'].mean()
print(f"Mean Chance of Admit for these applicants: {mean_chance_of_admit:.2f}")

Mean Chance of Admit for these applicants: 0.85


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [107]:
def standardize(column):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    mean = np.mean(column)
    std_dev = np.std(column)
    standardized_column = (column - mean) / std_dev
    return standardized_column

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [108]:
# your code here

# Create the standardized columns
admissions_predict_df['CGPA_std'] = standardize(admissions_predict_df['CGPA'])
admissions_predict_df['GRE_std'] = standardize(admissions_predict_df['GRE Score'])
admissions_predict_df['LOR_std'] = standardize(admissions_predict_df['LOR']) #important to remove the trailing spaces before
#otherwise it leads to error

admissions_predict_df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338


We will generate the decision choice at random using the code below. Please run the cell.

In [109]:
# Libraries
from random import choices

In [110]:
#std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

#decision_choice = choices(std_columns, k=admissions.shape[0])

std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']
decision_choice = choices(std_columns, k=admissions_predict_df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [111]:
# your code here
admissions_predict_df['deciding_column'] = decision_choice
admissions_predict_df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,LOR_std
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,CGPA_std
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,CGPA_std
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,LOR_std
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,GRE_std


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [112]:
# your code here

def decision(row):
    column_name = row['deciding_column']
    column_value = row[column_name]
    return 1 if column_value > 0.8 else 0

admissions_predict_df['Decision'] = admissions_predict_df.apply(decision, axis=1)

admissions_predict_df.head(10)



,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,Decision
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,LOR_std,1
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,CGPA_std,0
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,CGPA_std,0
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,LOR_std,0
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,GRE_std,1
5,321,8.20,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019,CGPA_std,0
6,308,7.90,101,2,3.0,4.0,0,0.68,-1.158723,-0.756445,0.635019,GRE_std,0
7,302,8.00,102,1,2.0,1.5,0,0.50,-0.992501,-1.276191,-2.155874,LOR_std,0
8,323,8.60,108,3,3.5,3.0,0,0.45,0.004836,0.542921,-0.481338,LOR_std,0
9,327,9.00,111,4,4.0,4.5,1,0.84,0.669726,0.889419,1.193197,GRE_std,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [113]:
# your code here
accepted_applicants = admissions_predict_df['Decision'].sum()
print("Number of accepted applicants:", accepted_applicants)

Number of accepted applicants: 88


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [134]:
#https://stackoverflow.com/questions/41476150/remove-or-replace-spaces-in-column-names
#https://stackoverflow.com/questions/46241120/how-to-remove-non-alpha-numeric-characters-from-strings-within-a-dataframe-colum

admissions_predict_df.columns = admissions_predict_df.columns.str.replace(' ', '_').str.lower()

admissions_predict_df.head(6)

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,LOR_std,1
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,CGPA_std,0
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,CGPA_std,0
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,LOR_std,0
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,GRE_std,1
5,321,8.20,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019,CGPA_std,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [140]:
# your code here

def boost_gre(row):
    if row['university_rating'] >= 4:
        return row['gre_score'] + 10
    return row['gre_score']

admissions_predict_df['adjusted_gre'] = admissions_predict_df.apply(boost_gre, axis=1)

admissions_predict_df['adjusted_gre'] = pd.cut(admissions_predict_df['gre_score'], bins=4)

admissions_predict_df.head(10)

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,LOR_std,1,"(327.5, 340.0]"
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,CGPA_std,0,"(315.0, 327.5]"
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,CGPA_std,0,"(315.0, 327.5]"
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,LOR_std,0,"(302.5, 315.0]"
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,GRE_std,1,"(327.5, 340.0]"
5,321,8.20,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019,CGPA_std,0,"(315.0, 327.5]"
6,308,7.90,101,2,3.0,4.0,0,0.68,-1.158723,-0.756445,0.635019,GRE_std,0,"(302.5, 315.0]"
7,302,8.00,102,1,2.0,1.5,0,0.50,-0.992501,-1.276191,-2.155874,LOR_std,0,"(289.95, 302.5]"
8,323,8.60,108,3,3.5,3.0,0,0.45,0.004836,0.542921,-0.481338,LOR_std,0,"(315.0, 327.5]"
9,327,9.00,111,4,4.0,4.5,1,0.84,0.669726,0.889419,1.193197,GRE_std,1,"(315.0, 327.5]"


In [139]:
unique_bins = admissions_predict_df['adjusted_gre'].unique()
print(unique_bins)

[(327.5, 340.0], (315.0, 327.5], (302.5, 315.0], (289.95, 302.5]]
Categories (4, interval[float64, right]): [(289.95, 302.5] < (302.5, 315.0] < (315.0, 327.5] < (327.5, 340.0]]
